# Heart Disease Dataset Analysis

Laura Flannigan, Kaitlyn Harvie, Hannah Morstead, & Tony Tang 

**Introduction**

   According to the World Health Organization, the leading cause of death worldwide remains as cardiovascular disease (2021). However, these diseases are preventable as research has revealed various risk factors in the development of heart disease. Yusuf et al. (2001) suggested that smoking, as well as high blood pressure and increased LDL cholesterol levels show causal links to cardiovascular disease. Additionally, it is widely accepted that a family history of heart disease (Kardian et al., 2003), as well as diabetes also increase risk  — however, the risk of diabetes differs in men and women (Wilson, 1998). 
    Using the Heart Disease Dataset from the Cleveland Clinic Foundation, we will determine how accurately a classification model, using criterion variables; age, sex, chest pain type, resting blood pressure, blood cholesterol levels, smoker status, family history of diabetes, family hisotry of cadiac disease, and presence of exercise-indudced angina, correctly predict the presence of heart disease for a new observation. 


**Preliminary Exploratory Data Analysis**

**Methods**

**Expected Outcomes and Significance**

References

World Health Organization. (2021). Cardiovascular Diseases (CVDs). Retreived from https://www.who.int/news-room/fact-sheets/detail/cardiovascular-diseases-(cvds) 

Yusuf, S., Reddy, S., Ounpuu, S. & Anand, S. (2001) Global Burden of Cardiovascular Diseases: Part I: General Considerations, the Epidemiologic Transition, Risk Factors, and Impact of Urbanization. Circulation, 104, 2746-2753. https://doi.org/10.1161/hc4601.099487

Kardia, S.L.R., Modell S. M., Peyser P. A. (2003). Family-Centered Approaches to Understanding and Preventing Coronary Heart Disease. American Journal of Preventive Medicine, 24, 143-151.  doi:10.1016/S0749-3797(02)00587-1 

Wilson, P. W. (1998). Diabetes mellitus and coronary heart disease. American Journal of Kidney Diseases, 32, 89-100. https://doi.org/10.1053/ajkd.1998.v32.pm9820468
